In [9]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [18]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [27]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [12]:
mlflow.set_experiment("EXP_5-ALL_ML_hpt_7000")

2025/09/26 13:06:28 INFO mlflow.tracking.fluent: Experiment with name 'EXP_5-ALL_ML_hpt_7000' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://apoorv-mlfow-bucket/602812102375773083', creation_time=1758872187297, experiment_id='602812102375773083', last_update_time=1758872187297, lifecycle_stage='active', name='EXP_5-ALL_ML_hpt_7000', tags={}>

In [29]:
# Map labels
y = df['category'].map({-1: 2, 0: 0, 1: 1})

# Split first, stratify ensures all classes in test
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    df['clean_comment'], y, test_size=0.2, stratify=y, random_state=42
)

# Vectorize
vectorizer = TfidfVectorizer(max_features=7000, ngram_range=(1,3))
X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Apply ADASYN only on train
adasyn = ADASYN(random_state=42)
X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)

# -----------------------------
# Step 4: MLflow logging function
# -----------------------------
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, best_params=None, study=None):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algo_name", model_name)

        # Log best hyperparameters
        if best_params:
            for param_name, param_value in best_params.items():
                mlflow.log_param(param_name, param_value)
        if study:
            mlflow.log_param("best_trial_number", study.best_trial.number)
            mlflow.log_metric("best_trial_value", study.best_trial.value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Metrics
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True, labels=[0,1,2])
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred, labels=[0,1,2])
        plt.figure(figsize=(8,6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=[0,1,2], yticklabels=[0,1,2])
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {model_name}")
        conf_matrix_file = f"confusion_matrix_{model_name}.png"
        plt.savefig(conf_matrix_file)
        mlflow.log_artifact(conf_matrix_file)
        plt.close()

        # Log model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# -----------------------------
# Step 5: Optuna objective for LightGBM
# -----------------------------
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# -----------------------------
# Step 6: Run Optuna and log best model
# -----------------------------
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    best_params = study.best_params
    best_model = LGBMClassifier(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        random_state=42
    )

    log_mlflow("LightGBM", best_model, X_train_vec, X_test_vec, y_train, y_test,
               best_params=best_params, study=study)

# -----------------------------
# Step 7: Run the experiment
# -----------------------------
run_optuna_experiment()

[I 2025-09-26 14:25:03,513] A new study created in memory with name: no-name-69f5eb22-ec25-4614-b9e9-038004128e02


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.398253 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 170580
[LightGBM] [Info] Number of data points in the train set: 36914, number of used features: 5446
[LightGBM] [Info] Start training from score -1.161819
[LightGBM] [Info] Start training from score -1.073625
[LightGBM] [Info] Start training from score -1.063295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

[I 2025-09-26 14:25:11,617] Trial 0 finished with value: 0.791763261966453 and parameters: {'n_estimators': 102, 'learning_rate': 0.09924284777459644, 'max_depth': 9}. Best is trial 0 with value: 0.791763261966453.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.388431 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 170580
[LightGBM] [Info] Number of data points in the train set: 36914, number of used features: 5446
[LightGBM] [Info] Start training from score -1.161819
[LightGBM] [Info] Start training from score -1.073625
[LightGBM] [Info] Start training from score -1.063295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

[I 2025-09-26 14:25:17,077] Trial 1 finished with value: 0.3440610936860766 and parameters: {'n_estimators': 58, 'learning_rate': 0.0008430392375131965, 'max_depth': 6}. Best is trial 0 with value: 0.791763261966453.


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.403709 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 170580
[LightGBM] [Info] Number of data points in the train set: 36914, number of used features: 5446
[LightGBM] [Info] Start training from score -1.161819
[LightGBM] [Info] Start training from score -1.073625
[LightGBM] [Info] Start training from score -1.063295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

[I 2025-09-26 14:31:24,187] Trial 29 finished with value: 0.7853538797218055 and parameters: {'n_estimators': 96, 'learning_rate': 0.098190048523955, 'max_depth': 9}. Best is trial 11 with value: 0.798309014046093.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.430090 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 170580
[LightGBM] [Info] Number of data points in the train set: 36914, number of used features: 5446
[LightGBM] [Info] Start training from score -1.161819
[LightGBM] [Info] Start training from score -1.073625
[LightGBM] [Info] Start training from score -1.063295
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

2025/09/26 14:32:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/26 14:32:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_ADASYN_TFIDF_Trigrams at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/602812102375773083/runs/91fb99504f514898976909c41bcba35b
🧪 View experiment at: http://ec2-16-171-17-119.eu-north-1.compute.amazonaws.com:5000/#/experiments/602812102375773083


In [20]:
import numpy as np
np.bincount(y_test)


array([2529, 3154], dtype=int64)

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_idx, test_idx in sss.split(df['clean_comment'], df['category']):
    X_train_raw = df['clean_comment'].iloc[train_idx]
    X_test_raw  = df['clean_comment'].iloc[test_idx]
    y_train     = df['category'].iloc[train_idx]
    y_test      = df['category'].iloc[test_idx]

print(y_test.value_counts())


category
1    3943
0    3161
Name: count, dtype: int64


In [28]:
df['category'].value_counts()

category
 1    15770
 0    12644
-1     8248
Name: count, dtype: int64